In [ ]:
import open3d as o3d
import numpy as np
import pandas as pd

# Load the .ply file
ply_file_path = r"C:\Users\Shashank\OneDrive\Desktop\Point Cloud\DATA\output\potholes59.ply"
point_cloud = o3d.io.read_point_cloud(ply_file_path)

# Visualize the point cloud (optional)
o3d.visualization.draw_geometries([point_cloud])

# Perform plane segmentation to find the ground plane
# Use RANSAC to detect the largest plane (assumed to be the ground)
plane_model, inliers = point_cloud.segment_plane(distance_threshold=0.01,
                                                 ransac_n=3,
                                                 num_iterations=1000)

[a, b, c, d] = plane_model  # Plane equation ax + by + cz + d = 0
ground_plane_points = point_cloud.select_by_index(inliers)
non_ground_points = point_cloud.select_by_index(inliers, invert=True)

# Calculate the depth of the pothole
# Extract the z-values (height) of the points not on the ground plane
pothole_z_values = np.asarray(non_ground_points.points)[:, 2]

# Calculate depth as the difference between the ground plane and pothole points
# Assuming the ground plane is relatively flat, we'll use its mean z-value
ground_z_values = np.asarray(ground_plane_points.points)[:, 2]
ground_z_mean = np.mean(ground_z_values)

# Depth of the pothole is how far below the ground plane the points are
pothole_depths = ground_z_mean - pothole_z_values

# Get the maximum depth (deepest point in the pothole)
max_depth = np.max(pothole_depths)

print(f"Estimated Maximum pothole depth: {max_depth:.4f} ")

# Optionally, visualize the segmented ground plane and pothole
ground_plane_points.paint_uniform_color([0, 1, 0])  # Green for ground
non_ground_points.paint_uniform_color([1, 0, 0])  # Red for pothole
o3d.visualization.draw_geometries([ground_plane_points, non_ground_points])

# Ground truth depth (if known, replace this with actual ground truth value)
ground_truth_depth = 0.15  # Example: replace with actual measured depth

# Calculate accuracy metrics if ground truth is available
if ground_truth_depth is not None:
    # Absolute error
    absolute_error = abs(max_depth - ground_truth_depth)
    
    # Relative error (percentage)
    relative_error = (absolute_error / ground_truth_depth) * 100
    
    print(f"Ground truth depth: {ground_truth_depth:.4f} meters")
    print(f"Absolute Error: {absolute_error:.4f} meters")
    print(f"Relative Error: {relative_error:.2f}%")

    # Step 1: Create a pandas DataFrame to display results in a tabular form
    results = {
        "Model": ["Pothole Depth Estimation"],
        "Estimated Depth (m)": [max_depth],
        "Ground Truth Depth (m)": [ground_truth_depth],
        "Absolute Error (m)": [absolute_error],
        "Relative Error (%)": [relative_error]
    }
    
    # Step 2: Convert the dictionary to a DataFrame
    results_df = pd.DataFrame(results)
    
    # Step 3: Display the DataFrame
    print("\nModel Accuracy Summary:")
    print(results_df)

    # Optionally, you can save this to a CSV file
    results_df.to_csv("pothole_depth_results.csv", index=False)



Estimated Maximum pothole depth: 0.3560 
Ground truth depth: 0.1500 meters
Absolute Error: 0.2060 meters
Relative Error: 137.32%

Model Results:
   Pothole Depth  Ground Truth Depth  Absolute Error  Relative Error (%)
0       0.355979                0.15        0.205979           137.31923
